This Notebook generates a 13 weeks long forecast of sales data for each product as part of the VN1 challenge. The underlying model 
used for the predictions is a LGBM with an Tweedie objective function. The predictions are generated using a recursive approach, 
where the model predicts the sales for the next week and then uses these values to generate the predictions for the next time period.

### Import Modules

In [1]:
import pandas as pd
import numpy as np
from functions_for_participants import *
import lightgbm as lgb
import tqdm as tq
import warnings


In [2]:
# Ignore Warnings
from pandas.errors import SettingWithCopyWarning
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)
warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

### Read Sales Data

Read Sales Data for both Phases and combine them into one single DataFrane


In [3]:
df_sales = pd.read_csv("Phase 0 - Sales.csv")
df_sales_1 = pd.read_csv("Phase 1 - Sales.csv")
df_sales = pd.concat([df_sales, df_sales_1], axis=1)
# Drop duplicate columns
df_sales = df_sales.loc[:, ~df_sales.columns.duplicated()]

In [4]:
# Melt the sales data into a Sales and Date columns
param = "Sales" 
list_columns = list(df_sales.columns)
list_dates_columns = list_columns [3:]
df_sales = df_sales.melt(id_vars=['Client', 'Warehouse', 'Product'], var_name='Date', value_vars=list_dates_columns, value_name=param)

In [5]:
df_sales

,Client,Warehouse,Product,Date,Sales
0,0,1,367,2020-07-06,7.0
1,0,1,639,2020-07-06,0.0
2,0,1,655,2020-07-06,21.0
3,0,1,1149,2020-07-06,7.0
4,0,1,1485,2020-07-06,0.0
...,...,...,...,...,...
2754694,46,318,13485,2024-01-01,0.0
2754695,46,318,13582,2024-01-01,67.0
2754696,46,318,13691,2024-01-01,2.0
2754697,46,318,13946,2024-01-01,0.0


In [6]:
# Specify the data types for the columns to optimize memory
dtype_dict = {
    'Client': 'int32',
    'Warehouse': 'int32',
    'Product': 'int32',
    "Sales": "float32",
}

# Convert the columns to the specified data types
for col in df_sales.columns:
    if col in dtype_dict.keys():
        df_sales[col] = df_sales[col].astype(dtype_dict[col])

### Derive Summary Statistics for Client, Warehouse, Product and Date Columns

This Section derives summary statistics(Sum,Mean,Std,Median) for the above mentioned columns. 


In [7]:
# List of grouping columns
elements = ['Client', 'Warehouse', 'Product',"Date"]

relationship_dict = {}

def combine_sales_data(df_combined, elements, relationship_dict):
    elements_dict = {}
    # Convert Date to datetime type in both DataFrames
    df_combined['Date'] = pd.to_datetime(df_combined['Date'])
    
    for element in elements:
        # Determine the columns to group by
        groupby_columns = [element] if element == 'Date' else [element, 'Date']
        
        # Perform group by on each combination along with 'Date'
        df_grouped = df_combined[groupby_columns + ["Sales"]].groupby(groupby_columns).agg(['sum', "std","mean","median"]).reset_index()
        
        # Remove multi-level index and rename columns
        df_grouped.columns = [
            f'total_{col[0]}_{col[1]}_by_{element}'.lower() if col[0] not in [element, 'Date'] else col[0]
            for col in df_grouped.columns
        ]
        columns = [col for col in df_grouped.columns if col not in [element, 'Date']]
        # print(columns)
        for column in columns:
            elements_dict[column] = element
            
        # Convert float columns to float32
        for column in columns:
            if df_grouped[column].dtype == 'float64':
                df_grouped[column] = df_grouped[column].astype('float32')
        
        # Merge with the original DataFrame
        df_combined = pd.merge(
            df_combined, 
            df_grouped,
            how='left', 
            on=groupby_columns
        )
    
    # Update relationship_dict with elements_dict
    relationship_dict.update(elements_dict)
    
    return df_combined, elements_dict

df_combined, elements_dict = combine_sales_data(df_sales, elements, relationship_dict)

In [8]:
df_combined

,Client,Warehouse,Product,Date,Sales,total_sales_sum_by_client,total_sales_std_by_client,total_sales_mean_by_client,total_sales_median_by_client,total_sales_sum_by_warehouse,...,total_sales_mean_by_warehouse,total_sales_median_by_warehouse,total_sales_sum_by_product,total_sales_std_by_product,total_sales_mean_by_product,total_sales_median_by_product,total_sales_sum_by_date,total_sales_std_by_date,total_sales_mean_by_date,total_sales_median_by_date
0,0,1,367,2020-07-06,7.0,46392.0,174.881729,49.883869,0.0,294.0,...,7.000000,7.0,245.0,53.265133,20.416666,7.0,105735.0,65.623100,7.024181,0.0
1,0,1,639,2020-07-06,0.0,46392.0,174.881729,49.883869,0.0,294.0,...,7.000000,7.0,0.0,0.000000,0.000000,0.0,105735.0,65.623100,7.024181,0.0
2,0,1,655,2020-07-06,21.0,46392.0,174.881729,49.883869,0.0,294.0,...,7.000000,7.0,910.0,209.894424,60.666668,7.0,105735.0,65.623100,7.024181,0.0
3,0,1,1149,2020-07-06,7.0,46392.0,174.881729,49.883869,0.0,294.0,...,7.000000,7.0,224.0,56.174728,22.400000,7.0,105735.0,65.623100,7.024181,0.0
4,0,1,1485,2020-07-06,0.0,46392.0,174.881729,49.883869,0.0,294.0,...,7.000000,7.0,0.0,0.000000,0.000000,0.0,105735.0,65.623100,7.024181,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2754694,46,318,13485,2024-01-01,0.0,2516.0,26.513712,10.752137,1.0,1804.0,...,26.925373,4.0,0.0,0.000000,0.000000,0.0,248037.0,96.780884,16.477579,0.0
2754695,46,318,13582,2024-01-01,67.0,2516.0,26.513712,10.752137,1.0,1804.0,...,26.925373,4.0,85.0,28.257742,17.000000,7.0,248037.0,96.780884,16.477579,0.0
2754696,46,318,13691,2024-01-01,2.0,2516.0,26.513712,10.752137,1.0,1804.0,...,26.925373,4.0,3.0,0.707107,1.500000,1.5,248037.0,96.780884,16.477579,0.0
2754697,46,318,13946,2024-01-01,0.0,2516.0,26.513712,10.752137,1.0,1804.0,...,26.925373,4.0,0.0,0.000000,0.000000,0.0,248037.0,96.780884,16.477579,0.0


In [9]:
### Fill missing values with 0
df_combined.fillna(0, inplace=True)

In [10]:
# Add Week, Month and Year columns
df_combined["Date"] = pd.to_datetime(df_combined["Date"])
df_combined["Week"] = df_combined["Date"].dt.isocalendar().week
df_combined["Month"] = df_combined["Date"].dt.month
df_combined["Year"] = df_combined["Date"].dt.year

### Create Lag Features

In [11]:
lagged_features = []

def create_lagged_features(df, columns_to_lag, groupby_columns,simple_lags):
    """
    Create lagged features and moving averages for specified columns in the DataFrame,
    grouped by specified columns.

    Parameters:
    df (pd.DataFrame): The input DataFrame.
    columns_to_lag (list): List of column names to create lagged features and moving averages for.
    groupby_columns (list): List of column names to group by.

    Returns:
    pd.DataFrame: DataFrame with lagged features and moving averages added.
    """
    new_columns = []

    new_columns = []
    grouped = df.groupby(groupby_columns)
    
    for col in columns_to_lag:
        # Create lagged features
        # print(col)
        for lag in simple_lags:
            lagged_col = f'{col}_lag_{lag}'
            # lagged_col_rocp = f'{col}_lag_{lag}_rocp'
            
            df[lagged_col] = grouped[col].shift(lag)
            # df[lagged_col_rocp] = np.log1p(grouped[col].shift(lag).pct_change())
            
            
            new_columns.append(lagged_col)
            
        # Create moving averages, ROCP, moving standard deviations
        # for window in [3, 5, 7, 9, 14, 20, 28]:
        #     ma_col = f'{col}_ma_{window}'
        #     rocp_col = f'{col}_ROCP_{window}'
        #     std_col = f'{col}_std_{window}'
        #     ema_col = f'{col}_ema_{window}'
            
        #     df[ma_col] = grouped[col].shift(1).rolling(window=window).mean()
        #     df[rocp_col] = grouped[col].pct_change(periods=window).shift(1)
        #     df[std_col] = grouped[col].shift(1).rolling(window=window).std()
        #     df[ema_col] = grouped[col].shift(1).ewm(span=window, adjust=False).mean()
            
        #     new_columns.extend([ma_col, rocp_col, std_col])

    
    df = df.copy()
    # Convert float columns to float32
    for column in new_columns:
        if df[column].dtype in [np.float64, np.float32]:
            if df[column].dtype == np.float64 and df[column].max() < np.finfo("float16").max:
                df[column] = df[column].astype("float16")
            elif df[column].dtype == np.float64 and df[column].max() < np.finfo("float32").max:
                df[column] = df[column].astype("float32")
            elif df[column].dtype == np.float32 and df[column].max() < np.finfo("float16").max:
                df[column] = df[column].astype("float16")
    return df, new_columns


columns_to_lag = [col for col in df_combined.columns if 'sales' in col.lower()]
groupby_columns = ["Client", "Warehouse", "Product"]
simple_lags = [i for i in range(1, 21)]
df_combined, new_columns = create_lagged_features(df_combined, columns_to_lag, groupby_columns,simple_lags)
lagged_features.extend(new_columns)

In [12]:
# Add Fourier terms
# Define the number of Fourier terms
num_terms = 13

# Create Fourier terms
for i in range(1, num_terms):  # Assuming n is defined somewhere in your code
    df_combined[f'sin_{i}'] = np.sin(2 * np.pi * i * df_combined['Week'] / 52).astype('float16')
    df_combined[f'cos_{i}'] = np.cos(2 * np.pi * i * df_combined['Week'] / 52).astype('float16')


df_combined = df_combined.fillna(0)
# Replace positive infinite values with 1 and negative infinite values with -1
df_combined = df_combined.replace([np.inf, -np.inf], [1, -1])

df_combined["product_week_warehouse"] = pd.Categorical(df_combined["Product"].astype(str) + "/" + df_combined["Week"].astype(str) + "/" + df_combined["Warehouse"].astype(str))

### Define Features to be used for Modelling

In [13]:
# Define the categorical features
cat_features = ['Client', 'Warehouse', 'Product',"Week","Month","Year"]

# Define the columns to drop from the training data
columns_to_drop = ["Date", "Sales"] + list(relationship_dict.keys())

# Get train columns
train_columns = df_combined.drop(columns=columns_to_drop).columns.tolist()

### Train LGMB Model Using Tweedie Loss

In [14]:
# Use the LightGBM model
params = {
    "objective": "tweedie",
    "metric": "rmse",
    "verbose": 1,
    "num_iterations": 100,
    "num_leaves": 256,
    "learning_rate": 0.2,
    "tweedie_variance_power": 1.1,
}

# Dictionary to store evaluation results
evals_result = {}

model = lgb.train(
    params,
    lgb.Dataset(
        df_combined[train_columns],
        label=df_combined["Sales"],
        categorical_feature=cat_features,
    ),
)

d:\miniconda\envs\forecasting\lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 2.031516 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 84767
[LightGBM] [Info] Number of data points in the train set: 2754699, number of used features: 351
[LightGBM] [Info] Start training from score 2.580259


### Check Feature Importance

In [15]:
importances = model.feature_importance(importance_type='split')
normalized_importances = (importances / importances.sum()) * 100
normalized_importances = normalized_importances.round(2)
list(sorted(zip(df_combined[train_columns].columns, normalized_importances), key=lambda xx: xx[1], reverse=True))

[('Product', 51.61),
 ('Week', 11.21),
 ('Warehouse', 6.93),
 ('Sales_lag_1', 1.55),
 ('Sales_lag_2', 1.05),
 ('Sales_lag_3', 0.67),
 ('Sales_lag_4', 0.59),
 ('total_sales_mean_by_warehouse_lag_1', 0.47),
 ('Client', 0.4),
 ('Sales_lag_5', 0.39),
 ('total_sales_sum_by_warehouse_lag_1', 0.34),
 ('total_sales_sum_by_date_lag_1', 0.34),
 ('Sales_lag_6', 0.29),
 ('total_sales_sum_by_client_lag_1', 0.29),
 ('total_sales_std_by_warehouse_lag_1', 0.29),
 ('total_sales_sum_by_product_lag_1', 0.29),
 ('Sales_lag_7', 0.27),
 ('Sales_lag_17', 0.25),
 ('total_sales_median_by_warehouse_lag_1', 0.25),
 ('Sales_lag_13', 0.24),
 ('total_sales_sum_by_product_lag_2', 0.24),
 ('total_sales_mean_by_product_lag_1', 0.24),
 ('Sales_lag_8', 0.23),
 ('Sales_lag_9', 0.23),
 ('total_sales_sum_by_client_lag_4', 0.22),
 ('total_sales_sum_by_client_lag_6', 0.22),
 ('total_sales_sum_by_client_lag_2', 0.21),
 ('total_sales_sum_by_product_lag_4', 0.21),
 ('Sales_lag_18', 0.2),
 ('total_sales_std_by_client_lag_1', 0.1

### Generate submission

Generate the submission by using **recursive forecasting**. This process involves forecasting the next week sales and using these values to predict the demand in the following time period. This process is repeated for 13 weeks.

In [16]:
horizons = {}
predictions_dict = {}
# Create 13 week horizon
for i in range(1, 14):
    horizons[i] = (df_combined["Date"].max() + pd.DateOffset(7*i))

In [17]:
horizons

{1: Timestamp('2024-01-08 00:00:00'),
 2: Timestamp('2024-01-15 00:00:00'),
 3: Timestamp('2024-01-22 00:00:00'),
 4: Timestamp('2024-01-29 00:00:00'),
 5: Timestamp('2024-02-05 00:00:00'),
 6: Timestamp('2024-02-12 00:00:00'),
 7: Timestamp('2024-02-19 00:00:00'),
 8: Timestamp('2024-02-26 00:00:00'),
 9: Timestamp('2024-03-04 00:00:00'),
 10: Timestamp('2024-03-11 00:00:00'),
 11: Timestamp('2024-03-18 00:00:00'),
 12: Timestamp('2024-03-25 00:00:00'),
 13: Timestamp('2024-04-01 00:00:00')}

In [18]:
# Columns to convert to object data type
convert_obj = ['Client', 'Warehouse', 'Product',"Sales"]

In [19]:
for time_window in tq.tqdm(horizons):
    print(horizons[time_window])
    
    # Take only the base data
    df_combined = df_combined[["Client", "Warehouse", "Product", "Date", "Sales"]]
    
    # Get Max 30 weeks before the horizon this speeds up the process immensely
    firts_date = horizons[time_window] - pd.DateOffset(weeks=30)
    df_combined = df_combined[(df_combined["Date"] >= firts_date) & (df_combined["Date"] <= horizons[time_window])]
    
    # Create the new time window and add it to the data
    df_cwp = df_combined[['Client', 'Warehouse', 'Product']].drop_duplicates().sort_values(by=['Client', 'Warehouse', 'Product'])
    df_cwp["Sales"] = 0
    df_cwp["Date"] = horizons[time_window]
    df_combined = pd.concat([df_combined, df_cwp], ignore_index=True)
    
    # Convert the columns to the specified data types
    for col in convert_obj:
        df_combined[col] = df_combined[col].astype("int32")
    
    # Combine sales data by client, warehouse, product and date    
    df_combined, elements_dict = combine_sales_data(df_combined, elements, relationship_dict)
    df_combined = df_combined.fillna(0)
    
    # Create lagged features
    df_combined, new_columns = create_lagged_features(df_combined, columns_to_lag, groupby_columns,simple_lags)

    # Create Date features
    df_combined["Date"] = pd.to_datetime(df_combined["Date"])
    df_combined["Week"] = df_combined["Date"].dt.isocalendar().week
    df_combined["Month"] = df_combined["Date"].dt.month
    df_combined["Year"] = df_combined["Date"].dt.year
    
    df_combined["product_week_warehouse"] = pd.Categorical(df_combined["Product"].astype(str) + "/" + df_combined["Week"].astype(str) + "/" + df_combined["Warehouse"].astype(str))
    

    # Create Fourier terms
    for i in range(1, num_terms):  # Assuming n is defined somewhere in your code
        df_combined[f'sin_{i}'] = np.sin(2 * np.pi * i * df_combined['Week'] / 52).astype('float16')
        df_combined[f'cos_{i}'] = np.cos(2 * np.pi * i * df_combined['Week'] / 52).astype('float16')
        
    # Predict the sales for the time window set all negative values to 0
    y_pred = model.predict(df_combined[train_columns][df_combined.Date == horizons[time_window]])
    y_pred = np.where(y_pred < 0, 0, y_pred)
    submission = y_pred
    
    # Store the predictions in the dictionary and update the combined DataFrame
    predictions_dict[horizons[time_window]] = submission
    df_combined.loc[df_combined.Date == horizons[time_window], "Sales"] = submission
    

  0%|          | 0/13 [00:00<?, ?it/s]

2024-01-08 00:00:00


  8%|▊         | 1/13 [00:09<01:57,  9.77s/it]

2024-01-15 00:00:00


 15%|█▌        | 2/13 [00:19<01:46,  9.67s/it]

...

 85%|████████▍ | 11/13 [01:44<00:18,  9.33s/it]

2024-03-25 00:00:00


 92%|█████████▏| 12/13 [01:53<00:09,  9.34s/it]

2024-04-01 00:00:00


100%|██████████| 13/13 [02:03<00:00,  9.47s/it]


### Get only the Predictions

In [20]:
df_cwp = df_combined[['Client', 'Warehouse', 'Product']].drop_duplicates().sort_values(by=['Client', 'Warehouse', 'Product'])
df_cwpd = pd.DataFrame(columns=["Client", "Warehouse", "Product", "Date", "Sales"])

for time_window in tq.tqdm(horizons):
    print(time_window)
    # dates_list = []
    df_cwp["Date"] = horizons[time_window]
    df_cwp["Sales"] = predictions_dict[horizons[time_window]]
    df_cwpd = pd.concat([df_cwpd, df_cwp], ignore_index=True)
df_cwpd["Sales"] = df_cwpd["Sales"].round()
df_cwpd["Date"] = pd.to_datetime(df_cwpd["Date"]).dt.date

  0%|          | 0/13 [00:00<?, ?it/s]

1
2
3
4
5
6
7
8
9
10
11


100%|██████████| 13/13 [00:00<00:00, 145.64it/s]

12
13


### Generate the submission file in the required format

In [21]:
pivoted_df = df_cwpd.pivot(index=['Client', 'Warehouse', 'Product'], columns='Date', values='Sales').reset_index()
pivoted_df.columns.name = None
### Save to csv
pivoted_df.to_csv("Submission_Phase_2_Teodor_Georgiev.csv", index=False)

### Check if my old submission is equal to the submission generated here

In [22]:
# pivoted_df_old = pd.read_csv("Submission_Phase_2.csv")
# pivoted_df_new = pd.read_csv("Submission_Phase_2_Teodor_Georgiev.csv")

In [23]:
# are_equal = pivoted_df_old.equals(pivoted_df_new)
# print("DataFrames are equal:", are_equal)

DataFrames are equal: True
